In [1]:
from bokeh.plotting import figure, output_notebook, show
import math
from bokeh.models.widgets import Select
from bokeh.models import CustomJS, ColumnDataSource
from bokeh.layouts import column
import itertools 

#self-defined module
import data_parser

output_notebook()
#import excel datasets, and convert them into dataframe
age_marriage_ds_df = data_parser.parse_excel_data('./data/indicator_age_of_marriage.xlsx', 'Data')

#Extract years and countries from the dataframe.
years = data_parser.extract_column_names(age_marriage_ds_df)
countries = data_parser.extract_row_names(age_marriage_ds_df)

#Convert dataframe into json format
data_json_list = data_parser.convert_df2json(age_marriage_ds_df, countries, years, "ages")

#Initialize the plot
p = figure(plot_width=600, plot_height=400)
p.title.text = 'Age at 1st marriage (women) among different countries'
p.xaxis.axis_label = "Year"
p.yaxis.axis_label = "Age"

#Move the data into datasource, which can be passed into customJS functions.
x = data_json_list[0]["years"]
y = data_json_list[0]["ages"]
source = ColumnDataSource(data=dict(x=x, y=y))
p.circle('x', 'y', source = source, size=10, alpha=0.5)
countries_source = ColumnDataSource(data=dict(countries=countries, data_json=data_json_list))

def callback(source=source, countries=countries_source, window=None):
    f = cb_obj.value
    ci = countries.data['countries'].index(f)
    source.data['x'] = countries.data['data_json'][ci]["years"] 
    source.data['y'] = countries.data['data_json'][ci]["ages"]
    source.change.emit()

select = Select(title="Countries:", value=countries[0], options=countries, callback=CustomJS.from_py_func(callback))
layout = column(select, p)


show(layout)



Loading BokehJS ...